# Introducción

En la siguiente notebook se presentará la consigna a seguir para el cuarto práctico del proyecto, correspondiente a la materia Aprendizaje Supervisado. El objetivo consiste en explorar la aplicación de diferentes métodos aprendidos en el curso, a través de experimentos reproducibles, y evaluando a su vez la conveniencia de uno u otro, así como la selección de diferentes hiperparámetros a partir del cálculo de las métricas pertinentes.

En este caso, enfrentamos un problema de clasificación binario de posicionamiento respecto de un tópico. Para este práctico vamos a utilizar únicamente los datos etiquetados, que ya vienen divididos en train y test. Buscamos analizar distintos problemas que puedan surgir como el desbalanceo de clases


## Organización

El trabajo va a estar organizado en dos grandes secciones: preprocesamiento y aplicación de los clasificadores. La sección de preprocesamiento va a ser muy similar (si no idéntica) que en el práctico de Introducción al Aprendizaje Automático. La mayoría del trabajo va a estar concentrada, por lo tanto, en la exploración de los clasificadores

#### Preprocesamiento
En la parte de preprocesamiento lo que vamos a hacer va a ser:

1 - Obtener el dataset

2 - Corpus de validación

3 - Tokenizar

4 - Aplicar alguna curación

5 - Balanceo de clases

6 - Representar el texto como vector: CountVectorizer

#### Clasificadores

1 - Support Vector Machines

2 - Random Forests

3 - Red Neuronal

4 - Evaluación de los clasificadores

5 - Optimización de Hiperparámetros

Esto para los tres datasets CON y SIN balanceo de clases


In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, make_scorer, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample

!pip install contractions
import contractions
import re
from sklearn import metrics as ms

# Preprocesamiento

### Cargamos los datos

In [ ]:
train = pd.read_csv("https://raw.githubusercontent.com/DamiFur/DiploDatos-StanceDetection/master/train.csv", sep=',', encoding="latin1").fillna(method="ffill")
test = pd.read_csv("https://raw.githubusercontent.com/DamiFur/DiploDatos-StanceDetection/master/test.csv", sep=',', encoding="latin1").fillna(method="ffill")

In [ ]:
abortion_train = train[train["Target"] == "Legalization of Abortion"]
abortion_test = test[test["Target"] == "Legalization of Abortion"]

climate_train = train[train["Target"] == "Climate Change is a Real Concern"]
climate_test = test[test["Target"] == "Climate Change is a Real Concern"]

feminism_train = train[train["Target"] == "Feminist Movement"]
feminism_test = test[test["Target"] == "Feminist Movement"]

In [ ]:
climate_train.head()

,Tweet,Target,Stance,Opinion Towards,Sentiment
613,"We cant deny it, its really happening. #SemST",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,other
614,RT @cderworiz: Timelines are short. Strategy m...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
615,SO EXCITING! Meaningful climate change action ...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
616,"Delivering good jobs for Albertans, maintainin...",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
617,@davidswann says he wants carbon fund to be sp...,Climate Change is a Real Concern,FAVOR,3. The tweet is not explicitly expressing opi...,other


Sólo vamos a usar el tweet y el stance. Como encima ya tenemos dividido el corpus según el target, vamos a eliminar todas las columnas excepto tweet y stance

In [ ]:
abortion_train.drop(columns = ["Target", "Opinion Towards", "Sentiment"], inplace=True)
abortion_test.drop(columns = ["Target", "Opinion Towards", "Sentiment"], inplace=True)

climate_train.drop(columns = ["Target", "Opinion Towards", "Sentiment"], inplace=True)
climate_test.drop(columns = ["Target", "Opinion Towards", "Sentiment"], inplace=True)

feminism_train.drop(columns = ["Target", "Opinion Towards", "Sentiment"], inplace=True)
feminism_test.drop(columns = ["Target", "Opinion Towards", "Sentiment"], inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### Tokenizamos

Utilizamos los parámetros seleccionados en el práctico anterior

In [ ]:
tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)

In [ ]:
tokenizer.tokenize(abortion_train["Tweet"].iloc[1])

['are',
 'you',
 'ok',
 'with',
 '#gop',
 'males',
 'telling',
 'you',
 'what',
 'you',
 'can',
 'and',
 "can't",
 'do',
 'with',
 'your',
 'own',
 'body',
 '?']

### Preprocesamiento

El preprocesamiento para twitter requiere tomar varias decisiones. Recomiendo que vean un poco el dataset y piensen con qué palabras quieren trabajar y cuales quieren remover. Les dejo el esqueleto de una función de preprocesamiento que sólo tokeniza pero que puede tomar dos parámetros optativos para remover hashtags y números.

#### Ejercicio 1

Agregarle a la función de preprocesamiento que borre las urls (palabras que empiecen con http). Agregarle código para que agregue o saque texto de acuerdo con al menos un criterio propuesto por ustedes (menciones a los usuarios, caritas/emojis, puntuación).

In [ ]:
def preprocesar(text, keep_hashtags=True, remove_numbers=True):
    
    text = contractions.fix(text) 
    
    toks = tokenizer.tokenize(text)
    
    ret = []
    for tok in toks:
        
        if re.match('https?://[A-Za-z0-9./]+',tok):
            continue

        if re.match('[A-Za-z0-9./]+@[A-Za-z0-9./]+',tok):
            continue
            
        if re.match(r'[^\w\s\#,]',tok): #emoji
            continue
            
        if re.match('[^?!.,]*[?.!,]$',tok): #puntuaciones
            continue
        
        if tok.isnumeric() and remove_numbers:
            continue
        ret.append(tok)
    return " ".join(ret)

In [ ]:
# Hacer esto para todos los datasets, train y test de los 3 tópicos
abortion_train["Tweet"] = abortion_train["Tweet"].apply(lambda x: preprocesar(x))
abortion_test["Tweet"] = abortion_test["Tweet"].apply(lambda x: preprocesar(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
abortion_train.head()

,Tweet,Stance
50,just laid down the law on abortion in my bioet...,AGAINST
51,are you ok with #gop males telling you what yo...,FAVOR
52,if you do not want your kid put it up for adop...,AGAINST
53,there should be a stigma to butchering pre-bor...,AGAINST
54,but is not that the problem then not enough fa...,NONE


### Balanceo de clases

Hacer subsampling del corpus de aborto y guardarlo como un nuevo dataset. A partir de ahora, todos los experimentos que corran deberán correrlos además de para los tres corpus respectivos a cada tópico, también para este nuevo corpus de aborto con sus clases balanceadas. Luego vamos a comparar los resultados obtenidos con y sin balanceo de clases

In [ ]:
train_abortion_against = abortion_train[abortion_train.Stance=="AGAINST"]
train_abortion_favor = abortion_train[abortion_train.Stance=="FAVOR"]
train_abortion_none = abortion_train[abortion_train.Stance=="NONE"]
   
train_abortion_subsampled = resample(train_abortion_against, 
                                      replace=True,  
                                      n_samples=len(train_abortion_favor),   
                                      random_state=123)

train_abortion_subsampled_none = resample(train_abortion_none, 
                                      replace=True,  
                                      n_samples=len(train_abortion_favor),   
                                      random_state=123)

abortion_train_balanced = pd.concat([train_abortion_subsampled, train_abortion_favor, train_abortion_subsampled_none])
abortion_train_balanced['Stance'].value_counts()

FAVOR      121
NONE       121
AGAINST    121
Name: Stance, dtype: int64

In [ ]:
abortion_train_balanced["Tweet"] = abortion_train_balanced["Tweet"].apply(lambda x: preprocesar(x))

In [ ]:
abortion_train_balanced.head()

,Tweet,Stance
2881,plenty of stem cells without baby smashing by ...,AGAINST
2486,i truly believe that #blacklivesmatter and tha...,AGAINST
2728,on my bday i am always so grateful to my bio p...,AGAINST
93,i was born months premature i am perfectly fin...,AGAINST
2464,preach black sheriff says if #blacklivesmatter...,AGAINST


# División del dataset en un corpus de validación

Para este práctico vamos a realizar la exploración de hiperparámetros utilizando un corpus de validación.

En el práctico anterior, hicieron una exploración manual de hiperparámetros probando distintos valores y tratando de analizar cómo cambian según se incrementen o decrementen los valores de los parámetros de cada modelo. Con esto pudieron obtener una idea general de cómo funcionaba cada uno, lo que les sirvió para elegir un modelo particular. Una vez elegido el modelo, buscaron los hiperparámetros óptimos para ese modelo de una manera más rigurosa, usando Grid Search (el cual utiliza Cross Validation: https://scikit-learn.org/stable/modules/cross_validation.html).

Sin embargo, los modelos utilizados en Aprendizaje Automático pueden tener cientos o hasta miles de parámetros con infinitos valores posibles, con lo cual, probarlos todos a mano es imposible. A la vez, el corpus utilizado como test no debe intervenir en el entrenamiento de nuestro modelo y sólo debe utilizarse como manera de medir su performance, o dicho en otras palabras, usar el corpus de test para buscar los mejores valores para los hiperparámetros lleva a nuestros modelos a hacer overfitting.

Usar Grid Search con Cross Validation puede ser una manera de solucionar este problema, pero debemos pasarle todos los valores de los parámetros que queremos que pruebe y debemos correr nuestro modelo una vez para cada combinación de hiperparámetros que querramos probar, lo cual puede tardar mucho.

En el último tiempo, se vienen usando cada vez más modelos iterativos, dentro de los cuales, las redes neuronales tienen una popularidad creciente. Los modelos iterativos van delineando la función de predicción por medio de aproximaciones sucesivas. Toman, por lo tanto, un nuevo parámetro que es la cantidad de iteraciones. A diferencia de otros hiperparámetros, las iteraciones son incrementales. Es decir, si mi modelo toma como parámetro 100 iteraciones, esto es equivalente a tomar el modelo con 99 iteraciones como parámetro y hacer una iteración más (siempre y cuando se use una misma semilla).

No tiene sentido, por lo tanto, al explorar los hiperparámetros, correr un modelo desde 0 hasta 99 iteraciones, probar con Cross Validation cuan bien funciona, luego correr desde 0 hasta 100 iteraciones, probar nuevamente como funciona, y así con todos los valores que querramos probar, dado que estaríamos repitiendo exponencialmente las iteraciones. Más bien, tiene sentido que entre iteración e iteración, probemos si nuestro modelo mejoró o no (o viene mejorando o no). Para esto utilizamos el corpus de validación.

El corpus de validación va a ser un pedazo de nuestro corpus de training que vamos a ir utilizando iteración a iteración para ver si nuestro modelo mejora o no y así saber cuando vamos a dejar de iterar. Esta técnica se llama *early stopping*.

Varios modelos (como el de Red Neuronal de keras que vamos a usar) ya tienen esta funcionalidad incorporada y directamente toman el corpus de validación como parámetro junto con una cantidad máxima de iteraciones. El modelo, por lo tanto, va a seguir iterando mientras que la evaluación en el corpus de validación sea mejor que en la iteración anterior o hasta que se alcancen la cantidad de iteraciones máxima.

En nuestro caso, el corpus de validación lo vamos a utilizar **sólo para la red neuronal**

### Ejercicio 1

Dividir el corpus de train en un corpus de train menor y otro de validación (20% del corpus de train). Hacer esto para cada uno de los datasets

In [ ]:
from sklearn.model_selection import train_test_split

ABORTO NO BALANCEADO

In [ ]:
train_abortion, validation_abortion = train_test_split(
    abortion_train, 
    test_size=0.2, 
    random_state=42
)

In [ ]:
len(train_abortion)

522

In [ ]:
y_test = abortion_test["Stance"]

In [ ]:
y_train = train_abortion["Stance"]

ABORTO BALANCEADO

In [ ]:
train_abortion_balanced, validation_abortion_balanced = train_test_split(
    abortion_train_balanced, 
    test_size=0.2, 
    random_state=42
)

In [ ]:
y_train_balanced = train_abortion_balanced["Stance"]

In [ ]:
len(train_abortion_balanced)

290

### Representación como vector

In [ ]:
def dataset_vectorizer(input_train, input_validation, input_test):
    text_train = input_train["Tweet"]
    text_validation = input_validation["Tweet"]
    text_test = input_test["Tweet"]

    vectorizer = CountVectorizer(
        binary=True, min_df=0.004, max_df=0.7, ngram_range=(1, 3)
    )

    X_result = vectorizer.fit_transform([*text_train, *text_validation, *text_test])

    VEC_train = X_result[:len(text_train)]
    VEC_validation = X_result[len(text_train):-len(text_test)]
    VEC_test = X_result[-len(text_test):]
    
    return VEC_train, VEC_validation, VEC_test

In [ ]:
VEC_train_abortion, VEC_validation_abortion, VEC_test_abortion = dataset_vectorizer(train_abortion, validation_abortion, abortion_test)

Hacer esto mismo para los otros tres datasets (Cambio climático, feminismo y el del aborto balanceado)

In [ ]:
VEC_train_abortion_balanced, VEC_validation_abortion_balanced, VEC_test_abortion_balanced = dataset_vectorizer(train_abortion_balanced, validation_abortion_balanced, abortion_test)

In [ ]:
# VEC_train_climate = #TODO
# VEC_test_climate = #TODO

# VEC_train_feminism = #TODO
# VEC_test_feminism = #TODO

#VEC_train_abortion_balanced = #TODO
# VEC_test_abortion_balanced = #TODO

# Clasificadores

A continuación van a realizar experimentos con tres clasificadores. Para cada uno van a tener que probar una serie de hiperparámetros. Les incluyo la documentación para que puedan leer qué es cada hiperparámetro que están probando. Luego de cada corrida, evaluan el clasificador con cuatro métricas: Accuracy Score, F1 micro, F1 macro y el promedio del F1 de la clase Favor con el F1 de la clase Against. La idea es que vayan cambiando los valores de un hiperparámetro dejando fijos el resto y vean cómo ese cambio impacta en las métricas. Finalmente, para cada clasificador escriban un pequeño informe planteando cuan sensible es cada parámetro respecto de cada métrica, por qué piensan que es así de sensible y cuales son los mejores valores que encontraron. Finalmente, elijan el clasificador que les parezca más adecuado para esta tarea y justifiquen su elección. Para ese clasificador que hayan elegido van a probar luego, una busqueda más exhaustiva de hiperparámetros usando Grid Search. Este procedimiento deben hacerlo **al menos** para **dos de los cuatro** datasets con los que venimos trabajando (aborto, aborto balanceado, cambio climático y feminismo). En el caso de la Red Neuronal, la cantidad de iteraciones óptima la van a encontrar usando el corpus de validación mediante *early stopping*.

### Support Vector Machines

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

ABORTO NO BALANCEADO

In [ ]:
# En principio, pueden utilizar el módulo que sigue, con los parámetros por defecto y los que definan a continuación:
C = 1.0
kernel = "rbf"
gamma = 0.1 # Esto lo tienen que cambiar a "degree" si usan el kernel polinomial
tol = 1e-3
class_weight = "balanced"

model = SVC(C = C, kernel = kernel, gamma=gamma, tol = tol, class_weight = class_weight)
model.fit(VEC_train_abortion, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_pred_test =  model.predict(VEC_test_abortion)
accuracy_test = accuracy_score(y_test, y_pred_test)
f1_test_micro = f1_score(y_test, y_pred_test, average="micro", labels=["NONE", "AGAINST", "FAVOR"])
f1_test_macro = f1_score(y_test, y_pred_test, average="macro", labels=["NONE", "AGAINST", "FAVOR"])
f1_test = f1_score(y_test, y_pred_test, average=None, labels=["NONE", "AGAINST", "FAVOR"])
# f1_test_average = #TODO
f1_test_average = (f1_test[1] + f1_test[2]) / 2

print("Accuracy para conjunto de test: %.2f" % accuracy_test)
print("F1 micro para conjunto de test: %.2f" % f1_test_micro)
print("F1 macro para conjunto de test: %.2f" % f1_test_macro)
print("F1 average para conjunto de test: %.2f" % f1_test_average)

Accuracy para conjunto de test: 0.61
F1 micro para conjunto de test: 0.61
F1 macro para conjunto de test: 0.49
F1 average para conjunto de test: 0.50


ABORTO BALANCEADO

In [ ]:
# En principio, pueden utilizar el módulo que sigue, con los parámetros por defecto y los que definan a continuación:
C = 1.0
kernel = "rbf"
gamma = 0.1 # Esto lo tienen que cambiar a "degree" si usan el kernel polinomial
tol = 1e-3
class_weight = "balanced"

model = SVC(C = C, kernel = kernel, gamma=gamma, tol = tol, class_weight = class_weight)
model.fit(VEC_train_abortion_balanced, y_train_balanced)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_pred_test =  model.predict(VEC_test_abortion_balanced)
accuracy_test = accuracy_score(y_test, y_pred_test)
f1_test_micro = f1_score(y_test, y_pred_test, average="micro", labels=["NONE", "AGAINST", "FAVOR"])
f1_test_macro = f1_score(y_test, y_pred_test, average="macro", labels=["NONE", "AGAINST", "FAVOR"])
f1_test = f1_score(y_test, y_pred_test, average=None, labels=["NONE", "AGAINST", "FAVOR"])
# f1_test_average = #TODO
f1_test_average = (f1_test[1] + f1_test[2]) / 2

print("Accuracy para conjunto de test: %.2f" % accuracy_test)
print("F1 micro para conjunto de test: %.2f" % f1_test_micro)
print("F1 macro para conjunto de test: %.2f" % f1_test_macro)
print("F1 average para conjunto de test: %.2f" % f1_test_average)

Accuracy para conjunto de test: 0.44
F1 micro para conjunto de test: 0.44
F1 macro para conjunto de test: 0.43
F1 average para conjunto de test: 0.44


### Random Forest

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

ABORTO NO BALANCEADO

In [ ]:
n_estimators = 7
#criterion = "gini"
#min_samples_split = 2
#min_samples_leaf = 1
#max_features = "log2"

model = RandomForestClassifier(random_state=2, 
                               n_estimators=n_estimators, 
                               #criterion = criterion, 
                               #min_samples_split=min_samples_split , 
                               #min_samples_leaf = min_samples_leaf,
                               #max_features = max_features, 
                               )
model.fit(VEC_train_abortion, train_abortion["Stance"])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=7,
                       n_jobs=None, oob_score=False, random_state=2, verbose=0,
                       warm_start=False)

In [ ]:
y_pred_test =  model.predict(VEC_test_abortion)
accuracy_test = accuracy_score(y_test, y_pred_test)
f1_test_micro = f1_score(y_test, y_pred_test, average="micro", labels=["NONE", "AGAINST", "FAVOR"])
f1_test_macro = f1_score(y_test, y_pred_test, average="macro", labels=["NONE", "AGAINST", "FAVOR"])
f1_test = f1_score(y_test, y_pred_test, average=None, labels=["NONE", "AGAINST", "FAVOR"])
# f1_test_average = #TODO
f1_test_average = (f1_test[1] + f1_test[2]) / 2

print("Accuracy para conjunto de test: %.2f" % accuracy_test)
print("F1 micro para conjunto de test: %.2f" % f1_test_micro)
print("F1 macro para conjunto de test: %.2f" % f1_test_macro)
print("F1 average para conjunto de test: %.2f" % f1_test_average)

Accuracy para conjunto de test: 0.64
F1 micro para conjunto de test: 0.64
F1 macro para conjunto de test: 0.56
F1 average para conjunto de test: 0.61


ABORTO BALANCEADO

In [ ]:
n_estimators = 39
criterion = "gini"
min_samples_split = 7
min_samples_leaf = 1
max_features = "auto"

model = RandomForestClassifier(random_state=2, 
                               n_estimators=n_estimators, 
                               criterion = criterion, 
                               min_samples_split=min_samples_split , 
                               min_samples_leaf = min_samples_leaf,
                               max_features = max_features, 
                               )
model.fit(VEC_train_abortion_balanced, train_abortion_balanced["Stance"])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=7,
                       min_weight_fraction_leaf=0.0, n_estimators=39,
                       n_jobs=None, oob_score=False, random_state=2, verbose=0,
                       warm_start=False)

In [ ]:
y_pred_test =  model.predict(VEC_test_abortion_balanced)
accuracy_test = accuracy_score(y_test, y_pred_test)
f1_test_micro = f1_score(y_test, y_pred_test, average="micro", labels=["NONE", "AGAINST", "FAVOR"])
f1_test_macro = f1_score(y_test, y_pred_test, average="macro", labels=["NONE", "AGAINST", "FAVOR"])
f1_test = f1_score(y_test, y_pred_test, average=None, labels=["NONE", "AGAINST", "FAVOR"])
# f1_test_average = #TODO
f1_test_average = (f1_test[1] + f1_test[2]) / 2

print("Accuracy para conjunto de test: %.2f" % accuracy_test)
print("F1 micro para conjunto de test: %.2f" % f1_test_micro)
print("F1 macro para conjunto de test: %.2f" % f1_test_macro)
print("F1 average para conjunto de test: %.2f" % f1_test_average)

Accuracy para conjunto de test: 0.47
F1 micro para conjunto de test: 0.47
F1 macro para conjunto de test: 0.46
F1 average para conjunto de test: 0.48


### Red Neuronal

En este caso vamos a usar la implementación de keras porque es sencillo utilizar el early stopping con el conjunto de validación.
https://keras.io/api/models/model_training_apis/



In [ ]:
# El output de nuestra red neuronal tiene que ser un valor numérico
codes = {}
codes["NONE"] = 0
codes["FAVOR"] = 1
codes["AGAINST"] = 2

def toNumeric(stance):
    return codes[stance]

labels = np.array(train_abortion["Stance"].apply(lambda x: toNumeric(x)))
validation = np.array(validation_abortion["Stance"].apply(lambda x: toNumeric(x)))

# labels = list(map(toNumeric, train_abortion["Stance"]))
# validation = list(map(toNumeric, validation_abortion["Stance"]))

Lo que deberían hacer acá es lo siguiente:

Deberían ir viendo como evlouciona la loss y el accuracy para el conjunto de validación tomando un epoch alto. Luego ver en qué epoch la loss empieza a subir o el accuracy empieza a bajar **en el corpus de validación**. Ver en qué epoch sucede eso y volver a entrenar sólo hasta ese epoch.

In [ ]:
input_dim = VEC_train_abortion.shape[1]

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD

alpha = 0.1
nodes_per_hidden_layer = 20
hidden_layer_depth = 1
activation = "softmax"# softmax, relu, tanh,
learning_rate = 1e-02

model = Sequential()
model.add(Dense(nodes_per_hidden_layer, input_dim=input_dim))

# Esto lo pueden usar si quieren agregar varias capas
# for i in range(hidden_layer_depth):
#     model.add(Dense(nodes_per_hidden_layer))

model.add(Activation(activation))

model.compile(loss='sparse_categorical_crossentropy', optimizer=SGD(lr=learning_rate), 
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                18880     
_________________________________________________________________
activation_1 (Activation)    (None, 20)                0         
Total params: 18,880
Trainable params: 18,880
Non-trainable params: 0
_________________________________________________________________


In [ ]:
VEC_validation_abortion.sort_indices()
VEC_train_abortion.sort_indices()

In [ ]:
model.fit(x=VEC_train_abortion, y=labels, batch_size=32, epochs=100, verbose=1, validation_data=(VEC_validation_abortion, validation))


Epoch 1/100
17/17 [==============================] - 0s 9ms/step - loss: 3.0191 - accuracy: 0.0517 - val_loss: 2.9574 - val_accuracy: 0.1145
Epoch 2/100
17/17 [==============================] - 0s 2ms/step - loss: 2.8938 - accuracy: 0.1475 - val_loss: 2.8268 - val_accuracy: 0.3053
Epoch 3/100
17/17 [==============================] - 0s 2ms/step - loss: 2.7737 - accuracy: 0.3238 - val_loss: 2.7010 - val_accuracy: 0.4580
Epoch 4/100
17/17 [==============================] - 0s 2ms/step - loss: 2.6583 - accuracy: 0.4444 - val_loss: 2.5811 - val_accuracy: 0.5420
Epoch 5/100
17/17 [==============================] - 0s 2ms/step - loss: 2.5485 - accuracy: 0.5096 - val_loss: 2.4661 - val_accuracy: 0.5725
Epoch 6/100
17/17 [==============================] - 0s 2ms/step - loss: 2.4442 - accuracy: 0.5307 - val_loss: 2.3593 - val_accuracy: 0.5878
Epoch 7/100
17/17 [==============================] - 0s 2ms/step - loss: 2.3466 - accuracy: 0.5268 - val_loss: 2.2605 - val_accuracy: 0.5878
Epoch 8/100
1

In [ ]:
VEC_test_abortion.shape, y_test.shape, y_pred_test.shape

NameError: ignored

In [ ]:
y_pred_test =  model.predict(VEC_test_abortion)
accuracy_test = accuracy_score(y_test, y_pred_test)
f1_test_micro = f1_score(y_test, y_pred_test, average="micro", labels=["NONE", "AGAINST", "FAVOR"])
f1_test_macro = f1_score(y_test, y_pred_test, average="macro", labels=["NONE", "AGAINST", "FAVOR"])
f1_test = f1_score(y_test, y_pred_test, average=None, labels=["NONE", "AGAINST", "FAVOR"])
# f1_test_average = #TODO
f1_test_average = (f1_test[1] + f1_test[2]) / 2

print("Accuracy para conjunto de test: %.2f" % accuracy_test)
print("F1 micro para conjunto de test: %.2f" % f1_test_micro)
print("F1 macro para conjunto de test: %.2f" % f1_test_macro)
print("F1 average para conjunto de test: %.2f" % f1_test_average)

print("Exactitud del algoritmo para conjunto de test: %.2f" % accuracy_test)

InvalidArgumentError: ignored

In [ ]:
y_pred_test.shape

### Grid Search

Nota: para usar Grid Search con la red neuronal de Keras tienen que usar un wrapper:
https://www.kaggle.com/shujunge/gridsearchcv-with-keras

In [ ]:

# Para la búsqueda de los mejores parámetros, por ejemplo de logistic regression, pueden usar:

exploring_params = {
        'C': [0.5, 1, 2, 5, 10, 20, 100, 200], # Inversa del coeficiente de regularización
        'max_iter': [1000, 5000, 10000],  # Cantidad de iteraciones
        'tol': [0.005, 0.002, 0.001, 0.0001]  # Precisión del algoritmo
    }

m = model
n_cross_val =  2 # Seleccionar folds
scoring = "f1_micro"
model = GridSearchCV(m, exploring_params, cv=n_cross_val, scoring=scoring)
#    model.fit(X_train, y_train)
    
model.fit(VEC_train_abortion, y_train)
# model.fit(VEC_train_abortion, labels3)

print("Mejor conjunto de parámetros:")
print(model.best_params_, end="\n\n")
print()
print("Puntajes de la grilla:", end="\n\n")
means = model.cv_results_['mean_test_score']
stds = model.cv_results_['std_test_score']
print()
print("Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):", end="\n\n")
y_true, y_pred = y_test, model.predict(VEC_test_abortion)
print(classification_report(y_true, y_pred), end="\n\n")

print("================================================", end="\n\n")

## Voting Classifier

A continuación se les pide implementar un Voting Classifier usando los modelos vistos hasta recién y analizar sus resultados

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
clf = VotingClassifier(estimators=[])

## Para el informe

Considerar el problema a utilizar y cual es la hipótesis que trabaja cada modelo. Describan las elecciones que tomaron: ¿utilizaron algún regularizador? ¿qué función de costo utilizaron y por qué la eligieron? ¿O en qué se diferencia esa función de costo de otras?